In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt

import skimage.color
import skimage.io

from config import Config
import utils
import model as modellib
import visualize
from model import log

C:\Users\tianh\Desktop\environments\mlenv\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
class ToyConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    # Give the configuration a recognizable name
    NAME = "toy"
    
    LEARNING_RATE = 1e-5
    WEIGHT_DECAY = 1e-6
    
    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 8

    # Number of classes (including background)
    NUM_CLASSES = 1 + 5  # background + 3 shapes

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 128
    IMAGE_MAX_DIM = 128

    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)  # anchor side in pixels

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 32

    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 100

    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 5

In [5]:
class InferenceConfig(ToyConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

In [7]:
class EasyAccess():
    
    def __init__(self):
        self.ROOT_DIR = os.getcwd()
        self.MODEL_DIR = os.path.join(self.ROOT_DIR, "logs")
        self.inference_config = InferenceConfig()
        self.model = modellib.MaskRCNN(mode="inference", 
                          config=self.inference_config,
                          model_dir=self.MODEL_DIR)
        self.model_path = model.find_last()[1]
        assert self.model_path != "", "Provide path to trained weights"
        print("Loading weights from ", self.model_path)
        self.model.load_weights(self.model_path, by_name=True)
        
    def detect(self, image):
        img = image
        if type(image) == str:
            img = skimage.io.imread(image)
        results = self.model.detect([img], verbose=1)